#### Monitor data

In [1]:
# !pip install faker
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [107]:
seed_value = 42
random.seed(seed_value)
faker = Faker()
Faker.seed(seed_value)
path = r"C:\Users\Sherry\Desktop\summer 2024\5310\project\csv"

In [12]:
# Generate data for employees
def generate_employees(num_employees=500, num_stores=5):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    employees = [{
        'EmployeeID': i,
        'FirstName': faker.first_name(),
        'LastName': faker.last_name(),
        'Position': random.choice(["Manager", "Cashier", "Clerk", "Butcher", "Janitorial Staff", "Security"]),
        'StoreID': random.randint(1, num_stores),  # Ensure store IDs are within the expected range
        'HireDate': faker.date_this_decade(),
        'Salary': round(random.uniform(30000.00, 120000.00), 2)
    } for i in range(1, num_employees + 1)]
    return pd.DataFrame(employees)

# Generating employee data
df_employees = generate_employees()
print(df_employees.head())

   EmployeeID FirstName LastName          Position  StoreID    HireDate  \
0           1  Danielle  Johnson           Butcher        1  2022-05-04   
1           2      John   Taylor  Janitorial Staff        3  2020-11-14   
2           3     Erica  Mcclain          Security        3  2020-09-27   
3           4  Brittany  Johnson          Security        4  2020-10-18   
4           5   Jeffery   Wagner  Janitorial Staff        1  2020-03-23   

      Salary  
0   51348.51  
1   81733.07  
2   31974.56  
3   54703.23  
4  111955.53  


In [13]:
df_employees.to_csv(path + "\employees.csv")

In [110]:
# Function to generate staff timeoff data
def generate_timeoffs(df_employees, num_timeoffs=200):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    timeoffs = []
    employee_ids = df_employees['EmployeeID'].tolist()
    random.shuffle(employee_ids)  # Get a list of all employee IDs

    for i in range(num_timeoffs):
        start_date = faker.date_this_year()  # Generate a start date within this year
        end_date = start_date + timedelta(days=random.randint(1, 14))  # Add 1-14 days to the start date

        timeoff = {
            'EmployeeID': employee_ids[i],
            'StartDate': start_date,
            'EndDate': end_date,
            'Reason': random.choice(["Vacation", "Sick", "Personal"]),
            'Status': random.choice(["Approved", "Rejected"])
        }
        timeoffs.append(timeoff)

    return pd.DataFrame(timeoffs)

# Generate data for staff timeoffs
df_timeoffs = generate_timeoffs(df_employees=df_employees, num_timeoffs=200)
print(df_timeoffs.head())

   EmployeeID   StartDate     EndDate    Reason    Status
0         273  2024-02-13  2024-02-22  Vacation  Approved
1         225  2024-01-10  2024-01-15  Vacation  Approved
2         390  2024-04-16  2024-04-17  Vacation  Rejected
3         366  2024-04-05  2024-04-15  Personal  Rejected
4         383  2024-03-27  2024-04-04  Vacation  Rejected


In [111]:
df_timeoffs.to_csv(path + "\staff_Timeoff.csv")

In [105]:
# Function to generate staff shifts
def generate_staff_shifts(df_employees, num_shifts=1000):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    staff_shifts = []
    employee_ids = df_employees['EmployeeID'].tolist()  # Get a list of all employee IDs

    for i in range(1, num_shifts + 1):
            shift_date = faker.date_between(start_date='-1y', end_date='today')  
            start_time = datetime.combine(shift_date, faker.time_object())  
            end_time = start_time + timedelta(hours=random.randint(1, 8)) 
            
            # Ensure end_time doesn't exceed the bounds of a day
            if end_time.day != start_time.day:
                end_time = start_time.replace(hour=23, minute=59, second=59)  # Set to last minute of the day if exceeding

            staff_shift = {
                'ShiftID': i,
                'EmployeeID': random.choice(employee_ids),
                'StartTime': start_time.strftime("%H:%M:%S"),  # Format time object to string
                'EndTime': end_time.strftime("%H:%M:%S"),  # Format time object to string
                'Date': shift_date.strftime("%Y-%m-%d")
            }
            staff_shifts.append(staff_shift)

    return pd.DataFrame(staff_shifts)

# Generate data for staff shifts
df_staff_shifts = generate_staff_shifts(df_employees=df_employees, num_shifts=1000)
print(df_staff_shifts.head())

   ShiftID  EmployeeID StartTime   EndTime        Date
0        1         397  03:34:23  11:34:23  2024-04-09
1        2         250  08:06:21  16:06:21  2023-08-14
2        3          83  19:40:56  21:40:56  2023-11-19
3        4         276  05:36:52  10:36:52  2023-10-30
4        5          23  12:45:07  18:45:07  2024-05-17


In [106]:
df_staff_shifts.to_csv(path + "\staff_shift.csv")

In [26]:
# Function to generate stores:
def generate_stores(df_employees, num_stores=5):
    faker.seed_instance(seed_value)
    stores = []

    for i in range(1, num_stores + 1):
        # Select a manager for each specific store
        possible_managers = df_employees[(df_employees['Position'] == 'Manager') & (df_employees['StoreID'] == i)]
        manager_id = random.choice(possible_managers['EmployeeID'].tolist()) if not possible_managers.empty else None

        store = {
            'StoreID': i,
            'StoreName': faker.company(),
            'Address': faker.street_address(),
            'City': 'New York',
            'State': 'NY',
            'ZipCode': faker.zipcode_in_state('NY'),
            'ManagerID': manager_id
        }
        stores.append(store)

    return pd.DataFrame(stores)

df_stores = generate_stores(df_employees=df_employees, num_stores=5)
print(df_stores.head())

   StoreID                        StoreName                      Address  \
0        1  Rodriguez, Figueroa and Sanchez           181 Johnson Course   
1        2                       Wagner Inc         863 Lawrence Valleys   
2        3                        Wolfe LLC  35116 Michael Key Suite 078   
3        4                    Miller-Carter   310 Kendra Common Apt. 164   
4        5        Flowers, Martin and Kelly    192 Frank Light Suite 835   

       City State ZipCode  ManagerID  
0  New York    NY   11792        299  
1  New York    NY   10054        105  
2  New York    NY   11023         47  
3  New York    NY   13715        141  
4  New York    NY   10459        442  


In [27]:
df_stores.to_csv(path + "\Stores.csv")

In [24]:
# Function to generate inventory data
def generate_inventory(df_stores, num_products=1000):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    store_ids = df_stores['StoreID'].tolist()  # Get a list of all store IDs

    inventory = [{
        'ProductID': i,
        'StoreID': random.choice(store_ids),  # Ensure only valid Store IDs are chosen
        'Quantity': random.randint(500, 5000),  # Random quantity between 0 and 500
        'UnitCost': round(random.uniform(5.00, 1000.00), 2)  # Random unit cost between $5 and $1000
    } for i in range(1, num_products + 1)]
    
    return pd.DataFrame(inventory)

# Generate data for inventory
df_inventory = generate_inventory(df_stores=df_stores, num_products=1000)
print(df_inventory.head())

   ProductID  StoreID  Quantity  UnitCost
0          1        4      1486    699.75
1          2        4      3085    690.76
2          3        1      2226    382.53
3          4        1      3165    364.79
4          5        3      2033    550.82


In [25]:
df_inventory.to_csv(path + "\Inventory.csv")

In [28]:
# Function to generate inventory change data
def generate_inventory_changes(df_inventory):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call

    inventory_changes = [{
        'ChangeID': i,
        'ProductID': row.ProductID,  # Use ProductID from the inventory using attribute access
        'StoreID': row.StoreID,      # Use StoreID associated with ProductID from the inventory using attribute access
        'ChangeType': random.choice(['Add', 'Remove']),  # Randomly choose type of inventory change
        'Quantity': random.randint(1, 100),  # Random quantity change
        'ChangeDate': faker.date_between(start_date='-1y', end_date='today')  # Random date for the inventory change
    } for i, row in enumerate(df_inventory.itertuples(), 1)]  # Generate as many entries as there are in the inventory
    
    return pd.DataFrame(inventory_changes)

# Generate data for inventory changes
df_inventory_changes = generate_inventory_changes(df_inventory=df_inventory)
print(df_inventory_changes.head())

   ChangeID  ProductID  StoreID ChangeType  Quantity  ChangeDate
0         1          1        4     Remove        99  2024-04-08
1         2          2        4        Add        32  2023-09-16
2         3          3        1     Remove        44  2023-08-13
3         4          4        1     Remove        65  2024-05-17
4         5          5        3        Add        11  2023-11-18


In [30]:
df_inventory_changes.to_csv(path + "\InventoryChanges.csv")

In [57]:
# Function to generate supplier data
def generate_suppliers(num_suppliers=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    suppliers = [{
        'VendorID': i,
        'VendorName': faker.company(),
        'Address': faker.street_address(),
        'City': faker.city(),
        'State': faker.state_abbr(),
        'ZipCode': faker.zipcode(),
        'Country': 'US'
    } for i in range(1, num_suppliers + 1)]
    
    return pd.DataFrame(suppliers)

# Generate data for suppliers
df_suppliers = generate_suppliers(num_suppliers=500)
print(df_suppliers.head())

   VendorID                       VendorName                         Address  \
0         1  Rodriguez, Figueroa and Sanchez              181 Johnson Course   
1         2                      Baldwin Ltd  79402 Peterson Drives Apt. 511   
2         3                        Blair PLC                1849 Ray Squares   
3         4                      James Group   55341 Amanda Gardens Apt. 764   
4         5                    Underwood LLC      1395 Diana Locks Suite 242   

              City State ZipCode Country  
0     East William    AK   74064      US  
1        Davisstad    PA   35172      US  
2  North Donnaport    CO   31013      US  
3        Lake Mark    WI   07832      US  
4        Lake Anna    KS   98413      US  


In [58]:
df_suppliers.to_csv(path + "\Suppliers.csv")

In [112]:
# Assuming df_suppliers is previously created and contains the supplier data
def generate_contacts(df_suppliers):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    contacts = [{
        'VendorID': supplier['VendorID'],  # Use VendorID from the suppliers DataFrame
        'Liaison': supplier['VendorName'],           # Random liaison name
        'Phone': faker.phone_number()      # Random phone number
    } for supplier in df_suppliers.to_dict('records')]  # Generate one contact entry per supplier
    
    return pd.DataFrame(contacts)

# Generate data for contacts
df_contacts = generate_contacts(df_suppliers=df_suppliers)
print(df_contacts.head())

   VendorID                          Liaison                 Phone
0         1  Rodriguez, Figueroa and Sanchez  +1-210-343-3218x1960
1         2                      Baldwin Ltd          538.990.8386
2         3                        Blair PLC      001-740-326-5423
3         4                      James Group          361-855-9407
4         5                    Underwood LLC          584-695-9310


In [34]:
df_contacts.to_csv(path + "\contact.csv")

In [37]:
# Assuming df_suppliers is the DataFrame that contains the supplier data
def generate_purchase_orders(df_suppliers, num_orders=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    vendor_ids = df_suppliers['VendorID'].tolist()  # Extract vendor IDs from the suppliers DataFrame

    purchase_orders = [{
        'PurchaseOrderID': i,
        'VendorID': random.choice(vendor_ids),  # Select a VendorID from the list of suppliers
        'OrderDate': (order_date := faker.date_between(start_date='-2y', end_date='today')),  # Assign and use OrderDate
        'ShippedDate': faker.date_between(start_date=order_date, end_date= 'today'),  # Ensure ShippedDate is after OrderDate
        'Status': random.choice(['Pending', 'Shipped', 'Cancelled', 'Delivered']),
        'TotalAmount': round(random.uniform(500.00, 20000.00), 2)  # Random total amount for the order
    } for i in range(1, num_orders + 1)]
    
    return pd.DataFrame(purchase_orders)

# Generate data for purchase orders
df_purchase_orders = generate_purchase_orders(df_suppliers=df_suppliers, num_orders=500)
print(df_purchase_orders.head())

   PurchaseOrderID  VendorID   OrderDate ShippedDate     Status  TotalAmount
0                1        75  2023-12-13  2024-01-25  Cancelled      1164.63
1                2        84  2022-08-23  2024-03-20  Delivered      1331.32
2                3        46  2023-03-05  2023-09-11    Shipped      9624.19
3                4        93  2023-01-24  2023-05-12  Delivered      5899.63
4                5        21  2024-02-27  2024-03-17  Cancelled     10262.45


In [38]:
df_purchase_orders.to_csv(path + "\PurchaseOrders.csv")

In [39]:
# Function to generate delivery data
def generate_deliveries(num_deliveries=100):
    faker.seed_instance(44)  # Resetting the seed for the faker instance each call
    deliveries = [{
        'Delivercompany_ID': i,
        'Company_Name': faker.company(),                # Random company name
        'Address': faker.street_address(),              # Random street address
        'City': faker.city(),                           # Random city
        'State': faker.state_abbr(),                    # Random state abbreviation
        'ZipCode': faker.zipcode(),                     # Random zipcode
        'Country': 'US',                     
        'Pricepercar': round(random.uniform(100.00, 1000.00), 2)  # Random price per car
    } for i in range(1, num_deliveries + 1)]
    
    return pd.DataFrame(deliveries)

# Generate data for deliveries
df_deliveries = generate_deliveries(num_deliveries=100)
print(df_deliveries.head())

   Delivercompany_ID                  Company_Name  \
0                  1                   Meyer-Miles   
1                  2    Gray, Sullivan and Vazquez   
2                  3                   Stark-Lopez   
3                  4  Cooley, Jensen and Mcpherson   
4                  5                    Lawson Inc   

                          Address          City State ZipCode Country  \
0             63403 Ashley Courts   Port Willie    ME   91400      US   
1      1549 Daniel Cliff Apt. 451    Austinfurt    CT   63059      US   
2  300 Mckinney Streets Suite 577  East Timothy    WY   97051      US   
3                2030 Vang Knolls  South Daniel    MA   15225      US   
4   97122 Christine Hill Apt. 410     Hillhaven    SC   89398      US   

   Pricepercar  
0       775.77  
1       506.35  
2       520.64  
3       240.31  
4       626.44  


In [40]:
df_deliveries.to_csv(path + "\Deliveries.csv")

In [113]:
# Function to generate customer data
def generate_customers(num_customers=500):
    faker.seed_instance(52)  # Resetting the seed for the faker instance each call
    customers = [{
        'CustomerID': i,
        'FirstName': faker.first_name(),
        'LastName': faker.last_name(),
        'Email': faker.email(),
        'Phone' : faker.phone_number(),
        'Address': faker.address()
    } for i in range(1, num_customers + 1)]
    
    return pd.DataFrame(customers)

# Generate data for customers
df_customers = generate_customers(num_customers=500)
print(df_customers.head())

   CustomerID FirstName   LastName                     Email  \
0           1   Tiffany      Brown        xwyatt@example.com   
1           2     Emily      Keith  douglasmeyer@example.net   
2           3  Kimberly  Hernandez    lorichavez@example.org   
3           4     Haley   Mcknight     deborah54@example.org   
4           5      Adam     Booker        padams@example.org   

                  Phone                                            Address  
0            8229625700         27347 Perry Walks\nNew Paulaland, OK 52276  
1  001-224-515-2696x903    2212 David Rapid Apt. 438\nJonesburgh, WA 93831  
2          286.783.1209  28219 Porter Skyway Apt. 247\nEast Amyberg, FL...  
3         (714)701-3385  26417 Lisa Causeway Suite 244\nLauraside, NJ 8...  
4    201.777.3149x04482          57135 Brian Pines\nPort William, ID 35239  


In [114]:
df_customers.to_csv(path + "\Customer.csv")

In [115]:
# Function to generate delivery order data
def generate_deliver_orders(df_stores, df_customers, num_orders=500):
    random.seed(44)
    faker.seed_instance(44)  # Resetting the seed for the faker instance each call
    store_ids = df_stores['StoreID'].tolist()  # Get a list of all store IDs
    customer_ids = df_customers['CustomerID'].tolist()  # Get a list of all customer IDs

    deliver_orders = [{
        'Deliverorder_ID': i,
        'Products': random.choice(['Dairy', 'Chocolate', 'Wine', 'Pasta', 'Sauce', 'Snakes', 'Fruit', 'Vegetable', 'Meat', 'Seafood']),  
        'Quantity': random.randint(500, 5000),  # Random quantity of products
        'Date': faker.date_between(start_date='-1y', end_date='today'),  # Random date for the order
        'StoreID': random.choice(store_ids),  # Randomly select a StoreID from the list
        'CustomerID': random.choice(customer_ids),  # Randomly select a CustomerID from the list
        'DeliveryFee': round(random.uniform(1000.00, 5000.00), 2)  # Random delivery fee
    } for i in range(1, num_orders + 1)]
    
    return pd.DataFrame(deliver_orders)

# Assuming df_stores and df_customers are previously created and contain the relevant data
df_deliver_orders = generate_deliver_orders(df_stores=df_stores, df_customers=df_customers, num_orders=500)
print(df_deliver_orders.head())

   Deliverorder_ID   Products  Quantity        Date  StoreID  CustomerID  \
0                1      Fruit      4760  2024-01-10        5         360   
1                2       Wine      3609  2024-02-22        2         149   
2                3  Chocolate       572  2024-03-02        1          81   
3                4      Sauce      3600  2024-05-03        5         208   
4                5     Snakes      3435  2024-07-04        1         174   

   DeliveryFee  
0      4448.15  
1      1116.39  
2      4097.79  
3      4064.38  
4      4366.09  


In [116]:
df_deliver_orders.to_csv(path + "\Deliver_Order.csv")

In [78]:
# Function to generate category data
def generate_categories(num_categories=10):
    faker.seed_instance(seed_value)
    cate = ['Dairy', 'Chocolate', 'Wine', 'Pasta', 'Sauce', 'Snakes', 'Fruit', 'Vegetable', 'Meat', 'Seafood']
    
    categories = [{
        'CategoryID': i,
        'Name': cate[i-1],  # Generate a simple, single-word category name and capitalize it
        'Description': faker.text(max_nb_chars=255)  # Generate a random text up to 255 characters as a description
    } for i in range(1, num_categories + 1)]
    
    return pd.DataFrame(categories)

# Generate data for categories
df_categories = generate_categories(num_categories=10)
print(df_categories.head())

   CategoryID       Name                                        Description
0           1      Dairy  Score each cause. Quality throughout beautiful...
1           2  Chocolate  Prove reduce raise author play. Rock clear her...
2           3       Wine  Seem shoulder future fall citizen about reveal...
3           4      Pasta  Best issue interest level. Pull worker better....
4           5      Sauce  Price north first end prove fire. How public f...


In [80]:
df_categories.to_csv(path + "\Categories.csv")

In [81]:
# Function to generate product data
def generate_products(df_categories, num_products=500):
    faker.seed_instance(46)  # Resetting the seed for the faker instance each call
    category_ids = df_categories['CategoryID'].tolist()  # Extract Category IDs from the categories DataFrame

    products = [{
        'ProductID': i,
        'Name': faker.word().capitalize(),  # Random product name
        'Description': faker.text(max_nb_chars=255),  # Random product description
        'Price': round(random.uniform(5.00, 500.00), 2),  # Random price between 5.00 and 500.00
        'CategoryID': random.choice(category_ids)  # Randomly select a CategoryID from the list
    } for i in range(1, num_products + 1)]
    
    return pd.DataFrame(products)

# Assuming df_categories is previously created and contains the category data
df_products = generate_products(df_categories=df_categories, num_products=500)
print(df_products.head())

   ProductID      Name                                        Description  \
0          1      View  Always only thought play charge Mrs. Air situa...   
1          2  Recently  Strategy result country hour scientist front o...   
2          3     Might  Family join woman truth summer avoid. Out deal...   
3          4      Lead  Somebody very it by radio small analysis. Stud...   
4          5     Party  Poor how wife foot team the plant. Drive inste...   

    Price  CategoryID  
0  116.32           3  
1  359.21           5  
2  224.08           6  
3    8.34           8  
4  136.16           7  


In [82]:
df_products.to_csv(path + "\Products.csv")

In [83]:
# Function to generate order data
def generate_orders(df_customers, num_orders=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    customer_ids = df_customers['CustomerID'].tolist()  # Extract Customer IDs from the customers DataFrame

    orders = [{
        'OrderID': i,
        'CustomerID': random.choice(customer_ids),  # Randomly select a CustomerID from the list
        'OrderDate': faker.date_between(start_date='-1y', end_date='today')  # Random date within the last year
    } for i in range(1, num_orders + 1)]
    
    return pd.DataFrame(orders)

# Assuming df_customers is previously created and contains the customer data
df_orders = generate_orders(df_customers=df_customers, num_orders=500)
print(df_orders.head())

   OrderID  CustomerID   OrderDate
0        1          23  2024-04-08
1        2         312  2023-09-16
2        3         307  2023-08-13
3        4         226  2024-05-17
4        5         287  2023-11-18


In [84]:
df_orders.to_csv(path + "\Orders.csv")

In [85]:
# Function to generate order details data
def generate_order_details(df_orders, df_products, num_order_details=1000):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    order_ids = df_orders['OrderID'].tolist()  # Extract Order IDs from the orders DataFrame
    product_ids = df_products['ProductID'].tolist()  # Extract Product IDs from the products DataFrame

    order_details = [{
        'OrderDetailID': i,
        'OrderID': random.choice(order_ids),  # Randomly select an OrderID from the list
        'ProductID': random.choice(product_ids),  # Randomly select a ProductID from the list
        'Quantity': random.randint(1, 10),  # Random quantity between 1 and 10
        'UnitPrice': round(random.uniform(5.00, 500.00), 2)  # Random unit price between 5.00 and 500.00
    } for i in range(1, num_order_details + 1)]
    
    return pd.DataFrame(order_details)

# Assuming df_orders and df_products are previously created and contain the relevant data
df_order_details = generate_order_details(df_orders=df_orders, df_products=df_products, num_order_details=1000)
print(df_order_details.head())

   OrderDetailID  OrderID  ProductID  Quantity  UnitPrice
0              1       20        479         3     437.32
1              2      349         66        10     108.30
2              3      434        395         4     126.45
3              4      377        397        10     211.61
4              5        3        276         5     305.30


In [86]:
df_order_details.to_csv(path + "\OrderDetails.csv")

In [120]:
# Function to generate sales transactions data
def generate_sales_transactions(df_orders, df_products, num_transactions=1000):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    store_ids = df_stores['StoreID'].tolist()
    order_ids = df_orders['OrderID'].tolist()  # Extract Order IDs from the orders DataFrame
    product_ids = df_products['ProductID'].tolist()  # Extract Product IDs from the products DataFrame

    transactions = [{
        'TransactionID': i,
        'StoreID': random.choice(store_ids),
        'OrderID': random.choice(order_ids),  # Randomly select an OrderID from the list
        'ProductID': random.choice(product_ids),  # Randomly select a ProductID from the list
        'TransactionDate': faker.date_between(start_date='-2y', end_date='today'),  # Random date within the last 2 years
        'TotalAmount': round(random.uniform(2000.00, 500000.00), 2)  # Random price between 20.00 and 500.00
    } for i in range(1, num_transactions + 1)]
    
    return pd.DataFrame(transactions)

# Assuming df_orders and df_products are previously created and contain the relevant data
df_transactions = generate_sales_transactions(df_orders=df_orders, df_products=df_products, num_transactions=1000)
print(df_transactions.head())

   TransactionID  StoreID  OrderID  ProductID TransactionDate  TotalAmount
0              1        2      156        159      2023-12-14    451412.69
1              2        3      219        318      2022-10-30     23269.71
2              3        5      366         25      2022-08-24    454258.60
3              4        4      351        168      2024-03-02    435317.42
4              5        2      261         20      2023-03-06    457625.65


In [122]:
df_transactions.to_csv(path + "\SalesTransactions.csv")

In [89]:
# Function to generate membership subscriptions data
def generate_membership_subscriptions(df_customers, num_subscriptions=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    customer_ids = df_customers['CustomerID'].tolist()  # Extract Customer IDs from the customers DataFrame

    subscriptions = [{
        'SubscriptionID': i,
        'CustomerID': random.choice(customer_ids),  # Randomly select a CustomerID from the list
        'SubscriptionType': random.choice(['Basic', 'Premium', 'VIP']),  # Randomly select a subscription type
        'StartDate': (startdate := faker.date_between(start_date='-2y', end_date='today')),  # Random start date within the last 2 years
        'EndDate': faker.date_between(start_date=startdate, end_date='+1y'),  # End date should be after start date, within the next year
        'SubscriptionFee': round(random.uniform(10.00, 200.00), 2)  # Random subscription fee between 10.00 and 200.00
    } for i in range(1, num_subscriptions + 1)]
    
    return pd.DataFrame(subscriptions)

# Assuming df_customers is previously created and contains the customer data
df_subscriptions = generate_membership_subscriptions(df_customers=df_customers, num_subscriptions=500)
print(df_subscriptions.head())

   SubscriptionID  CustomerID SubscriptionType   StartDate     EndDate  \
0               1         183              VIP  2023-12-13  2024-03-08   
1               2          33              VIP  2022-08-23  2023-10-24   
2               3         275              VIP  2023-02-10  2024-01-22   
3               4         418            Basic  2022-11-20  2023-04-28   
4               5          26              VIP  2024-01-11  2025-03-09   

   SubscriptionFee  
0           137.45  
1           175.45  
2           116.66  
3           165.82  
4            61.47  


In [90]:
df_subscriptions.to_csv(path + "\MembershipSubscriptions.csv")

In [91]:
# Function to generate product review data
def generate_product_reviews(df_products, df_customers, num_reviews=1000):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    product_ids = df_products['ProductID'].tolist()  # Extract Product IDs from the products DataFrame
    customer_ids = df_customers['CustomerID'].tolist()  # Extract Customer IDs from the customers DataFrame

    product_reviews = [{
        'ReviewID': i,
        'ProductID': random.choice(product_ids),  # Randomly select a ProductID from the list
        'CustomerID': random.choice(customer_ids),  # Randomly select a CustomerID from the list
        'Rating': random.randint(1, 5),  # Random rating between 1 and 5
        'ReviewText': faker.text(),  # Random review text
        'ReviewDate': faker.date_between(start_date='-2y', end_date='today')  # Random date within the last 2 years
    } for i in range(1, num_reviews + 1)]
    
    return pd.DataFrame(product_reviews)

# Assuming df_products and df_customers are previously created and contain the relevant data
df_product_reviews = generate_product_reviews(df_products=df_products, df_customers=df_customers, num_reviews=1000)
print(df_product_reviews.head())

   ReviewID  ProductID  CustomerID  Rating  \
0         1        259         308       5   
1         2        335         486       2   
2         3         82          29       3   
3         4        261          96       5   
4         5        150         454       1   

                                          ReviewText  ReviewDate  
0  Score each cause. Quality throughout beautiful...  2023-10-06  
1  Peace technology officer relate animal directi...  2023-05-12  
2  Meeting before another body. Civil quite other...  2024-03-16  
3  Campaign little near enter. Institution deep m...  2023-04-24  
4  Court movie cell contain leg. Kitchen technolo...  2024-05-27  


In [92]:
df_product_reviews.to_csv(path + "\ProductReviews.csv")

In [94]:
# Function to generate sales products combo data
def generate_sales_products_combo(df_products, num_combos=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    product_ids = df_products['ProductID'].tolist()  # Extract Product IDs from the products DataFrame

    sales_products_combo = [{
        'ComboID': i,
        'ProductID1': random.choice(product_ids),  # Randomly select a ProductID for the first product
        'ProductID2': random.choice(product_ids),  # Randomly select a ProductID for the second product
        'NumberofSales': random.randint(1, 500),  # Random number of sales between 1 and 500
        'TotalRevenue': round(random.uniform(1000.00, 50000.00), 2)  # Random total revenue between 1000.00 and 50000.00
    } for i in range(1, num_combos + 1)]
    
    return pd.DataFrame(sales_products_combo)

# Assuming df_products is previously created and contains the product data
df_sales_products_combo = generate_sales_products_combo(df_products=df_products, num_combos=500)
print(df_sales_products_combo.head())

   ComboID  ProductID1  ProductID2  NumberofSales  TotalRevenue
0        1         415         103            297      13145.26
1        2         470         157            373       4059.28
2        3         476         490            219      22032.42
3        4          98         207             44       6891.79
4        5         167         338            217      48936.15


In [95]:
df_sales_products_combo.to_csv(path + "\SalesProductsCombo.csv")

In [99]:
# Function to generate revenue data
def generate_revenues(df_stores, num_revenues=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    store_ids = df_stores['StoreID'].tolist()  # Extract Store IDs from the stores DataFrame

    revenues = [{
        'RevenueID': i,
        'StoreID': random.choice(store_ids),  # Randomly select a StoreID from the list
        'Month': faker.date_this_month(),  # Random date within the current month for simplicity
        'Amount': round(random.uniform(10000.00, 500000.00), 2)  # Random revenue amount between 1000.00 and 50000.00
    } for i in range(1, num_revenues + 1)]
    
    return pd.DataFrame(revenues)

# Assuming df_stores is previously created and contains the store data
df_revenues = generate_revenues(df_stores=df_stores, num_revenues=500)
print(df_revenues.head())

   RevenueID  StoreID       Month     Amount
0          1        4  2024-08-02   63073.82
1          2        1  2024-08-01  167042.96
2          3        3  2024-08-01   79828.04
3          4        1  2024-08-03  378478.80
4          5        5  2024-08-01  482796.29


In [100]:
df_revenues.to_csv(path + "\Revenue.csv")

In [103]:
# Function to generate expense data
def generate_expenses(df_stores, num_expenses=500):
    faker.seed_instance(seed_value)  # Resetting the seed for the faker instance each call
    store_ids = df_stores['StoreID'].tolist()  # Extract Store IDs from the stores DataFrame

    expenses = [{
        'ExpenseID': i,
        'StoreID': random.choice(store_ids),  # Randomly select a StoreID from the list
        'Month': faker.date_this_month(),  # Random date within the current month for simplicity
        'ExpenseType': random.choice(['Utilities', 'Rent', 'Supplies', 'Salaries', 'Maintenance']),  # Random expense type
        'Amount': round(random.uniform(5000.00, 200000.00), 2)  # Random expense amount between 500.00 and 20000.00
    } for i in range(1, num_expenses + 1)]
    
    return pd.DataFrame(expenses)

# Assuming df_stores is previously created and contains the store data
df_expenses = generate_expenses(df_stores=df_stores, num_expenses=500)
print(df_expenses.head())

   ExpenseID  StoreID       Month  ExpenseType    Amount
0          1        1  2024-08-02    Utilities  25556.22
1          2        2  2024-08-01         Rent  72146.16
2          3        1  2024-08-01     Salaries  36608.31
3          4        3  2024-08-03     Supplies   8589.82
4          5        5  2024-08-01  Maintenance  28272.18


In [104]:
df_expenses.to_csv(path + "\Expense.csv")

In [59]:
# Define insertion queries for each table in the specified order
insert_queries = {
    'Employee': '''
        INSERT INTO Employee (EmployeeID, FirstName, LastName, Position, StoreID, HireDate, Salary)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''',
    'staff_timeoff': '''
        INSERT INTO staff_timeoff (EmployeeID, StartDate, EndDate, Reason, Status)
        VALUES (%s, %s, %s, %s, %s)
    ''',
    'Stores': '''
    INSERT INTO Stores (StoreID, StoreName, Address, City, State, ZipCode, EmployeeID)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''',
    'staff_shift': '''
        INSERT INTO staff_shift (ShiftID, EmployeeID, StartTime, EndTime, Date)
        VALUES (%s, %s, %s, %s, %s)
    ''',
    'Inventory': '''
        INSERT INTO Inventory (ProductID, StoreID, Quantity, UnitCost)
        VALUES (%s, %s, %s, %s)
    ''',
    'InventoryChanges': '''
        INSERT INTO InventoryChanges (ChangeID, ProductID, StoreID, ChangeType, Quantity, ChangeDate)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''',
    'Suppliers': '''
        INSERT INTO Suppliers (VendorID, VendorName, Address, City, State, ZipCode, Country)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''',
    'contact': '''
        INSERT INTO contact (VendorID, Liaison, Phone)
        VALUES (%s, %s, %s)
    ''',
    'PurchaseOrders': '''
        INSERT INTO PurchaseOrders (PurchaseOrderID, VendorID, OrderDate, ShippedDate, Status, TotalAmount)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''',
    'Deliveries': '''
        INSERT INTO Deliveries (Delivercompany_ID, Company_Name, Address, City, State, ZipCode, Country, Pricepercar)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ''',
    'Customer': '''
        INSERT INTO Customer (CustomerID, FirstName, LastName, Email, Phone, Address)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''',
    'Deliver_Order': '''
        INSERT INTO Deliver_Order (Deliverorder_ID, Products, Quantity, Date, StoreID, CustomerID, DeliveryFee)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''',
    'Categories': '''
        INSERT INTO Categories (CategoryID, Name, Description)
        VALUES (%s, %s, %s)
    ''',
    'Products': '''
        INSERT INTO Products (ProductID, Name, Description, Price, CategoryID)
        VALUES (%s, %s, %s, %s, %s)
    ''',
    'Orders': '''
        INSERT INTO Orders (OrderID, CustomerID, OrderDate)
        VALUES (%s, %s, %s)
    ''',
    'OrderDetails': '''
        INSERT INTO OrderDetails (OrderDetailID, OrderID, ProductID, Quantity, UnitPrice)
        VALUES (%s, %s, %s, %s, %s)
    ''',
    'SalesTransactions': '''
        INSERT INTO SalesTransactions (TransactionID, StoreID, OrderID, TransactionDate, TotalAmount)
        VALUES (%s, %s, %s, %s, %s)
    ''',
    'MembershipSubscriptions': '''
        INSERT INTO MembershipSubscriptions (SubscriptionID, CustomerID, SubscriptionType, StartDate, EndDate, SubscriptionFee)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''',
    'ProductReviews': '''
        INSERT INTO ProductReviews (ReviewID, ProductID, CustomerID, Rating, ReviewText, ReviewDate)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''',
    'SalesProductsCombo': '''
        INSERT INTO SalesProductsCombo (ComboID, ProductID1, ProductID2, NumberofSales, TotalRevenue)
        VALUES (%s, %s, %s, %s, %s)
    ''',
    'Revenue': '''
        INSERT INTO Revenue (RevenueID, StoreID, Month, Amount)
        VALUES (%s, %s, %s, %s)
    ''',
    'Expense': '''
        INSERT INTO Expense (ExpenseID, StoreID, Month, ExpenseType, Amount)
        VALUES (%s, %s, %s, %s, %s)
    '''
}


In [60]:
def insert_data():
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    try:
        # Insert data into each table
        for table, query in insert_queries.items():
            print(table)
            # Select the corresponding DataFrame
            df = globals().get(f'df_{table.lower()}')
            if df is not None:
                # Insert data from DataFrame into the table
                for record in df.itertuples(index=False, name=None):
                    cursor.execute(query, record)
                print(f"Data inserted into {table} table.")
        
        # Commit the changes
        conn.commit()
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

if __name__ == "__main__":
    insert_data()


Employee
staff_timeoff
Stores
Error inserting data: insert or update on table "stores" violates foreign key constraint "stores_employeeid_fkey"
DETAIL:  Key (employeeid)=(1) is not present in table "employee".

